In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt

# 1. Tải và tiền xử lý dữ liệu MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0  # Chuẩn hóa về [0, 1]
x_test = x_test.astype('float32') / 255.0
x_train = np.expand_dims(x_train, axis=-1)  # Thêm chiều kênh: (60000, 28, 28, 1)
x_test = np.expand_dims(x_test, axis=-1)    # (10000, 28, 28, 1)

# 2. Xây dựng mạng CNN cơ bản để trích xuất embedding
def create_base_network():
    inputs = layers.Input(shape=(28, 28, 1))
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(64, name='embedding')(x)  # Embedding 64 chiều
    x = layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(x)  # Chuẩn hóa L2
    return Model(inputs, x)

base_network = create_base_network()
base_network.summary()

# 3. Hàm tạo triplet
def create_triplets(x, y, num_triplets=1000):
    triplets = []
    for _ in range(num_triplets):
        # Chọn ngẫu nhiên Anchor và Positive (cùng nhãn)
        anchor_idx = np.random.randint(0, len(x))
        positive_idx = np.random.choice(np.where(y == y[anchor_idx])[0])
        # Chọn Negative (khác nhãn)
        negative_idx = np.random.choice(np.where(y != y[anchor_idx])[0])
        triplets.append((x[anchor_idx], x[positive_idx], x[negative_idx]))
    return triplets

# Tạo tập triplet từ dữ liệu huấn luyện
triplets = create_triplets(x_train, y_train, num_triplets=5000)
anchor_images = np.array([t[0] for t in triplets])
positive_images = np.array([t[1] for t in triplets])
negative_images = np.array([t[2] for t in triplets])

# 4. Định nghĩa Triplet Loss
def triplet_loss(y_true, y_pred, alpha=0.2):
    anchor, positive, negative = y_pred[:, 0:64], y_pred[:, 64:128], y_pred[:, 128:]
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)
    loss = tf.maximum(pos_dist - neg_dist + alpha, 0.0)
    return tf.reduce_mean(loss)

# 5. Xây dựng mô hình triplet
anchor_input = layers.Input(shape=(28, 28, 1), name='anchor')
positive_input = layers.Input(shape=(28, 28, 1), name='positive')
negative_input = layers.Input(shape=(28, 28, 1), name='negative')

anchor_embedding = base_network(anchor_input)
positive_embedding = base_network(positive_input)
negative_embedding = base_network(negative_input)

merged_output = layers.Concatenate(axis=1)([anchor_embedding, positive_embedding, negative_embedding])
triplet_model = Model(inputs=[anchor_input, positive_input, negative_input], outputs=merged_output)
triplet_model.compile(optimizer='adam', loss=triplet_loss)

# 6. Huấn luyện mô hình
triplet_model.fit(
    [anchor_images, positive_images, negative_images],
    np.zeros((len(anchor_images),)),  # y_true không cần thiết, chỉ để khớp API
    epochs=10,
    batch_size=32,
    verbose=1
)

# 7. Lưu embedding của tập tham chiếu (reference set)
reference_images = x_train[:100]  # Lấy 100 mẫu làm tham chiếu
reference_labels = y_train[:100]
reference_embeddings = base_network.predict(reference_images)

# 8. Nhận dạng dữ liệu mới mà không cần huấn luyện lại
def recognize_digit(new_image, reference_embeddings, reference_labels):
    new_embedding = base_network.predict(np.expand_dims(new_image, axis=0))
    distances = euclidean_distances(new_embedding, reference_embeddings)[0]
    nearest_idx = np.argmin(distances)
    predicted_label = reference_labels[nearest_idx]
    return predicted_label, distances[nearest_idx]

# Thử nhận dạng một mẫu mới từ tập test
new_image = x_test[0]
true_label = y_test[0]
predicted_label, distance = recognize_digit(new_image, reference_embeddings, reference_labels)
print(f"True Label: {true_label}, Predicted Label: {predicted_label}, Distance: {distance:.4f}")

# 9. Trực quan hóa
plt.imshow(new_image.squeeze(), cmap='gray')
plt.title(f"True: {true_label}, Predicted: {predicted_label}")
plt.show()